In [1]:
import os

In [2]:
%pwd

'c:\\Users\\USER\\OneDrive\\Desktop\\Projects\\KD-disease-classidication\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\USER\\OneDrive\\Desktop\\Projects\\KD-disease-classidication'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="harchandanipiyush48"
os.environ["MLFLOW_TRACKING_PASSWORD"]="26060456"

In [6]:
import dagshub
dagshub.init(repo_owner='harchandanipiyush48', repo_name='KD-disease-classidication', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as harchandanipiyush48

Initialized MLflow to track repo "harchandanipiyush48/KD-disease-classidication"

Repository harchandanipiyush48/KD-disease-classidication initialized!

🏃 View run gregarious-ram-649 at: https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow/#/experiments/0/runs/3c047bbe684c40fa9e51558202ed4c6c
🧪 View experiment at: https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow/#/experiments/0


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self):
        """Prepare validation data generator."""
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        """Load a trained Keras model from disk."""
        return tf.keras.models.load_model(path)

    def evaluation(self):
        """Load model, run evaluation, and save results."""
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        """Save evaluation scores to a JSON file."""
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Log params & metrics
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            try:
                if tracking_url_type_store != "file" and "dagshub" not in mlflow.get_tracking_uri():
                    # ✅ Local MLflow server with registry
                    mlflow.keras.log_model(
                        self.model,
                        artifact_path="model",
                        registered_model_name="VGG16Model"
                    )
                else:
                    # ✅ DagsHub or file store → save model manually as artifact
                    local_path = Path("saved_model")
                    mlflow.keras.save_model(self.model, path=local_path)
                    mlflow.log_artifacts(str(local_path), artifact_path="model")

            except Exception as e:
                print(f"⚠️ Model logging fallback due to error: {e}")
                local_path = Path("saved_model")
                mlflow.keras.save_model(self.model, path=local_path)
                mlflow.log_artifacts(str(local_path), artifact_path="model")


In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 2207 images belonging to 2 classes.
138/138 ━━━━━━━━━━━━━━━━━━━━ 921s 7s/step - accuracy: 0.7295 - loss: 0.5301


2025/09/25 14:31:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/09/25 14:31:27 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


⚠️ Model logging fallback due to error: 'WindowsPath' object has no attribute 'startswith'
🏃 View run persistent-robin-86 at: https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow/#/experiments/0/runs/f6e2c826074d45ba9a203903a30e4151
🧪 View experiment at: https://dagshub.com/harchandanipiyush48/KD-disease-classidication.mlflow/#/experiments/0


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'saved_model\\data'